In [2]:
import hilbertcurve
import nglview
import numpy as np
import py3Dmol
from hilbertcurve.hilbertcurve import HilbertCurve

import pytraj as pt 
import matplotlib.pyplot as plt
from matplotlib import cm


In [3]:
class DummyPDB:
    temp = "ATOM      1  Du  TMP     1       0.000   0.000   0.000  1.00  0.00";   # public 
    tmpout = "tmp.pdb"
    def __init__(self, vecX, vecY, vecZ, elems=""):
        self.x=vecX
        self.y=vecY
        self.z=vecZ
        self.elems = elems
        self.PDBlines()
        if len(self.x) != len(self.y) or len(self.x) != len(self.z):
            print("Length of dimention is not equil")
            
    def PDBlines(self):
        theline=""
        for i in range(len(self.x)):
            tmpcopy=self.temp;
            indlen = len(str(i));
            atmInd = "{:>5}".format(i+1)
            X = "{:>8}".format(self.x[i])
            Y = "{:>8}".format(self.y[i])
            Z = "{:>8}".format(self.z[i])
            try:
                Elem = self.elems[i];
            except:
                Elem = "Du";
            Elem = "{:>4}".format(Elem);
            tmpcopy = tmpcopy[:12]+Elem+tmpcopy[16:]
            tmpcopy = tmpcopy[:6]+atmInd+tmpcopy[11:]
            tmpcopy = tmpcopy[:30]+X+tmpcopy[38:]
            tmpcopy = tmpcopy[:38]+Y+tmpcopy[46:]
            tmpcopy = tmpcopy[:46]+Z+tmpcopy[54:]
            theline += tmpcopy+"\n"
        self.lines = theline;
        with open(self.tmpout,'w') as outfile1:
            outfile1.write(self.lines)
        return theline
    def show(self):
        viewer = nglview.show_file(self.tmpout)
        return viewer


In [4]:
with open("rec.pdb","r") as file1:
    pdbfile = [i.strip('\n') for i in  file1.readlines()] #.split('\n')
with open("trajLigand.mol2","r") as file2:
    molfile = [i.strip('\n') for i in file2.readlines()]
    
ite=4; 
dim=3;
cutoff=14;
distances = list(range(2**(ite*dim)))

HC1 = HilbertCurve(ite, dim)
points1 = HC1.points_from_distances(distances)

HC2 = HilbertCurve(6,2)
points2 = HC2.points_from_distances(distances)


In [5]:
traj = pt.load("rec.pdb")
traj.top.set_reference(traj[0])

proXYZ = traj.xyz[0][traj.top.select(":1-183")]
ovcenter = np.mean(traj.xyz[0][traj.top.select(":LIG <: 5 & !:5")], axis=0)
ligcenter = np.mean(traj.xyz[0][traj.top.select(":LIG")], axis = 0)


print(ovcenter)
print(ligcenter)



[ 0.5461     -0.95677931  6.48768621]
[ 0.3896     -0.31993333  6.30606667]


In [36]:
tmp=traj.top.atoms
select1 = traj.top.select(":LIG <@ 8 & !:LIG, T3P & !@CL,K ")
center = np.mean(traj.xyz[0][select1])
tmpelems = np.array([i.name for i in tmp])[select1]
tmpxyz = traj.xyz[0][select1]
tmpx = tmpxyz[:,0]
tmpy = tmpxyz[:,1]
tmpz = tmpxyz[:,2]

points1 = np.array(points1)
points2 = np.array(points2)

x2 = points2[:,0]
y2 = points2[:,1] 

pointT = (points1 / 16 * 8 ) + (ovcenter-3)



HCx = np.round(pointT[:,0], 4)
HCy = pointT[:,1].round(4)
HCz = np.round(pointT[:,2], 4)

dummy=DummyPDB(HCx,HCy,HCz)

In [45]:
view = nglview.show_file("tmp.pdb")
view.add_component("trajLigand.mol2")
# view.add_component("rec.pdb")
view.clear_representations()
view.add_representation("point", color="red")
view



NGLWidget()

In [ ]:
class PDBparser:
    def __init__():
        self.Coor = 

a="abcdef"

res = "{:>5}".format(a)
print(res)

In [28]:
a=np.array([1.67787878, 32.84783, 1.6778, 1.67782564,1.67787894])
print(a.round(3))
print(np.round_(a,4))


[ 1.678 32.848  1.678  1.678  1.678]
[ 1.6779 32.8478  1.6778  1.6778  1.6779]
